jupyter nbconvert UAD_MO.ipynb --to slides --no-prompt --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_input_tags remove_input --post serve  --SlidesExporter.reveal_theme=white --SlidesExporter.reveal_scroll=True --SlidesExporter.reveal_transition=slide 

<center>
<img align="center" src="IMG\LZ4.png">
</center>

# Математическое моделирование распределения легкой примеси в приземном слое атмосферы, описанное в рамках теории подобия 

## Численное решение

### Импорт пакетов

In [1]:
import os
import math
from numba import njit
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import pickle

### Функции теории подобия МО

In [2]:
def psi(z, L, z0):
 #   return 1./(1.+4.7*(z+z0)) if L>0 else (1+ (1 if L>0 else -1)*15*z) ** 0.25
    return 1. / (1. + 4.7*(z+z0)) if L>0 else (1 + 15*z) ** 0.25


def iks(z, L):
 #   return (1+ (1 if L>0 else -1)*15*z) ** 0.25
    return (1 + 15*z) ** 0.25


def f_MO(z, L):
    return (np.log(z*L) + 4.7*z if L>0 else 
            (np.log(z*abs(L)) - 2.*np.log(0.5*(1+iks(z,L))) -
            np.log(0.5*(1+iks(z,L))) + 2.* np.arctan(iks(z,L))))


def u_MO(z, L, z0):
    return 1./KAPPA * (f_MO(z,L)-f_MO(z0,L))


def k_MO(z, L, z0):
    return KAPPA*(z+z0)*psi(z,L,z0)

### Решение конечно-разностной системы уравнений методом  прогонки

In [3]:
@njit
def progon(A, C, B, F, Y):

    alfa = np.zeros(n + 1)
    beta = np.zeros(n + 1)

    # прямой ход метода прогонки
    # вычисление коэффициентов прогонки
    alfa[0] = B[0] / C[0]
    beta[0] = F[0] / C[0]
    for i in range(1, n + 1):
        alfa[i] = B[i] / (C[i] - alfa[i-1]*A[i])
        beta[i] = (A[i]*beta[i-1] + F[i]) / (C[i] - alfa[i-1]*A[i])

    # обратный ход метода прогонки
    Y[i] = beta[n]
    for i in range(n - 1, -1, -1):
        Y[i] = alfa[i]*Y[i+1] + beta[i]

### Вычисление по явной схеме

In [4]:
@njit
def explicit_scheme():
    # вычисление по явной схеме
    for j in range(m):
        for i in range(1, n-1):
            sigma = dx / (uf[i] * dz**2)
            k_minus = 0.5*(kf[i] + kf[i-1])
            k_plus = 0.5*(kf[i] + kf[i+1])
            #c[i, j+1] = (sigma * k_plus * c[i+1, j] + (1- sigma * (k_minus + k_plus)) * c[i, j]
             #           + sigma * k_minus * c[i-1, j])
            c[i, j+1] = (sigma * (k_plus*c[i+1, j] + k_minus*c[i-1, j]) +
                           (1 - sigma*(k_minus+k_plus)) * c[i, j])

        c[0, j+1] = c[1, j+1]
        c[n, j] = 0

    for j in range(1, m):
        for i in range(n):
            cpoint[i,j] = c[i,j] / (2 * math.sqrt(math.pi*k0*x[j]))
    # вычисление по явной схеме выполнено

### Вычисление по неявной схеме

In [5]:
@njit
def implicit_scheme(cn):
    #global cn
    # определение массивов для коэффициентов системы 
    # для метода прогонки в чисто неявной схеме
    A = np.zeros(n+1)
    C = np.zeros(n+1)
    B = np.zeros(n+1)
    F = np.zeros(n+1)
    # вычисление по чисто неявной схеме
    for j in range(1, m+1):
        # вычисление коэффциентов системы уравнений
        C[0], B[0], F[0] = 1., 1., 0.
        C[n], A[n], F[n] = 1., 0., 0.
        for i in range(1,n):
            sigma = uf[i] * dz**2 / dx
            k_minus = 0.5 * (kf[i] + kf[i-1])
            k_plus = 0.5 * (kf[i] + kf[i+1])
            A[i] = k_minus
            C[i] = k_plus + k_minus + sigma
            B[i] = k_plus
            F[i] = sigma * cn[i, j-1] #*0.983
        progon(A, C, B, F, cn[:, j])
    # вычисление по неявной схеме выполнено

### Вычисление по схеме с весами $\sigma=0,5$

In [6]:
@njit
def Krank_Nikolson():
    # вычисление по схеме Кранка-Николсона
    for j in range(1, m+1):
        # вычисление коэффциентов системы уравнений
        C[0] = 1.; B[0] = 1.; F[0] = 0.
        C[n] = 1.; A[n] = 0.; F[n] = 0.
        for i in range(1,n):
            sigma = uf[i] * dz**2 / dx
            k_minus = 0.5 * (kf[i] + kf[i-1])
            k_plus = 0.5 * (kf[i] + kf[i+1])
            A[i] = k_minus
            C[i] = k_plus + k_minus + 2*sigma
            B[i] = k_plus
            F[i] = (2*sigma*ck[i,j-1] + k_plus*(ck[i+1,j-1]-ck[i,j-1])
            - k_minus*(ck[i,j-1]-ck[i-1,j-1]))


        progon(A, C, B, F, ck[:, j])

# вычисление по схеме Кранка-Николсона выполнено

### Задаем входные параметры модели

In [7]:
# Флаг, определяющий, как рассчитаваются u и k
# flag_analit = True, если они константы 
flag_analit = False 

In [8]:
# Постоянная Кармана
KAPPA = 0.4187
# Масштаб длины Монина-Обухова
L = -100.
# Источник мощностью M на высоте H
M = 10.
if flag_analit:
    # Задача с постоянными значениями
    # скорости ветра и коэффициента турбулентности.
    # Рассматриваем сравнение с аналитическим решением.
    # Безразмерная высота соответствует источнику на высоте 10 м
    H = 10. / abs(L)
    U = 10.
    K = 5.
    # Скорость ветра на высоте источника равна U.
    #uH = U
    #C = M / (uH*abs(L))
    UDIN = 0.1407  # Переход к безразмерным переменным
    C = M / (UDIN*abs(L))
else:  
    # Задача с профилями скорости ветра и коэффициента
    # турбулентности, определяемыми в соответствии
    # с теорией подобия МО.
    # Безразмерная высота
    H = 0.2
    # Динамическая скорость
    UDIN = 0.1407   # Stable Breedt et al.
    #UDIN = 0.1439   # Neutral Breedt et al.
    #UDIN = 0.3739   # Unstable Breedt et al.
    # Коэффициент для перехода к безразмерной концентрации
    C = M / (UDIN*abs(L))

### Определяем границы расчетной области, шаг и сетку

In [9]:
# Шаг сетки в безразмерных переменных
dz = 0.005
dx = 0.01
# Правая граница расчетной области в метрах
right = 3000.

# Безразмерная правая граница
xmax = right / abs(L)   # + 0.00001

# Безразмерный параметр шероховатости
z0 = 0.0002
# Верхняя граница расчетной области
zmax = 1. + z0
# Массив значений x и z
#x = np.arange(0.001, xmax+dx, dx)
x = np.arange(0., xmax+dx, dx)
z = np.arange(z0, zmax+dz, dz)
# Количество шагов
n = z.size - 1
m = x.size - 1

### Определяем массивы значений коэффициентов турбулентности

In [10]:
if flag_analit:
    # Скорость ветра и коэффициент турбулентности
    # задаем постоянными.
    uf = np.ones(n+1)
    kf = np.ones(n+1)
    uf = uf * U / UDIN
    kf = kf * K / (UDIN*L)
else:
    # Скорость ветра и коэффициент турбулентности
    # определяем с помощью функций в рамках теории подобия.
    kf = np.zeros(n)
    uf = np.zeros(n)
    kf = k_MO(z, L, z0)
    uf = u_MO(z, L, z0)

In [11]:
# массив cn - решение задачи для линейного источника в неявной задаче
cn = np.zeros((n+1, m+1))

### Задание начального условия

In [12]:
# Определяем номер узла, соответствующего источнику
i = 1
while i <= n :
    if z[i-1] <= H and H <= z[i]:
        break
    i += 1

iH = i
# И задаем начальное условие в этом узле
#cn[iH, 0] = M/(uH * dz)
cn[iH, 0] = 1./(uf[iH]*dz)

### Решение по неявной схеме

In [13]:
implicit_scheme(cn)

### Функция графического вывода результатов

In [14]:
def mv_graphics_stat(graphs_visible, my_title,
                     mv_title_x, mv_title_y,
                     file_name,
                     xaxis_min, xaxis_max,
                     mv_legend_x, mv_legend_anchor):
    fig = go.Figure(data = graphs_visible,
                    layout_xaxis_range = [xaxis_min,
                                          xaxis_max])
    fig.layout.font.family = 'Times'
    fig.layout.font.size = 14
    
    fig.update_layout(xaxis_title = mv_title_x,
                  yaxis_title = mv_title_y,
                  legend=dict(x = mv_legend_x,
                              xanchor =
                              mv_legend_anchor),
                  autosize=False,
                  width=600, height=400)
    fig.update_layout(margin = 
                      dict(l=25, r=0, t=0, b=25))
    fig.write_image("IMG/" + file_name + ".png")    
    fig.show()

In [15]:
flag_output = True
if flag_output:
    # save the output profiles to a file
    if not os.path.exists("UAD_data"):
        os.mkdir("UAD_data")

    g_data_file = open("UAD_data\data_"+str(H)+"_"+str(L)+"_"+str(z0), "wb")
    pickle.dump(H, g_data_file)
    pickle.dump(L, g_data_file)
    pickle.dump(z0, g_data_file)
    pickle.dump(C, g_data_file)

    pickle.dump(x, g_data_file)
    pickle.dump(z, g_data_file)
    pickle.dump(cn, g_data_file)

    g_data_file.close()

In [26]:
graphs_visible = [go.Scatter(visible = True, x=x*abs(L), y=cn[iH-1,:]*C, 
                               name='{:,.1f} м'.format(z[iH-1]*abs(L)).replace(",", " "),
                               line_dash = 'dot', line_color = '#6e0cb3'),
                  go.Scatter(visible = True, x=x*abs(L), y=cn[iH+1,:]*C, 
                               name='{:,.1f} м'.format(z[iH+1]*abs(L)).replace(",", " "),
                               line_dash = 'solid', line_color = '#6e0cb3'),

                 ]


my_title = "Приземные концентрации на разных высотах"
mv_title_x = "$$x, м$$"
mv_title_y = "$$c^{\prime}$$"

file_name = "data_"+str(H)+"_"+str(L)+"_"+str(z0)
xaxis_min = 0.
xaxis_max = 300.
mv_legend_x = 1.
mv_legend_anchor = "right"

mv_graphics_stat(graphs_visible, my_title,
                 mv_title_x, mv_title_y,
                 file_name,
                 xaxis_min, xaxis_max,
                 mv_legend_x, mv_legend_anchor)


## Графический анализ результатов

In [28]:
print('L = ', L)
print('H = ', H*abs(L))

L =  -100.0
H =  20.0


<center>
<img align="center" src="IMG\data_0.2_-100.0_0.0002.png">
</center>

### Аналитическое решение задачи
Для $u,k=\text{const}$
$$
c=\frac{M}{2\sqrt{\pi k_{z}ux}}\times\left\{ \exp\left[-\frac{u\left(z+H\right)^{2}}{4k_{z}x}\right]+\exp\left[-\frac{u\left(z-H\right)^{2}}{4k_{z}x}\right]\right\} 
$$


In [17]:
if flag_analit:
    # массив c_an - аналитическое решение задачи для  U,K=const
    c_an = np.zeros((n+1, m+1))
    for j in range(1, m+1):
        c_an[:,j] = (M / (2*np.sqrt(math.pi*x[j]*abs(L)*K*U))
              * (np.exp(-U * (z*abs(L) + H*abs(L))**2 / (4*K*x[j]*abs(L)))
               + np.exp(-U * (z*abs(L) - H*abs(L))**2 / (4*K*x[j]*abs(L)))))

In [18]:
if flag_analit:
    graphs_visible = [go.Scatter(visible = True, x=x[:]*abs(L), y=cn[iH+1,:]*C, 
                                   name='Численное решение',
                                   line_dash = 'solid', line_color = '#6e0cb3'),
                       go.Scatter(visible = True, x=x[1:]*abs(L), y=c_an[iH+1,1:], 
                                   name='Аналитическое решение',
                                   line_dash = 'dot', line_color = '#6e0cb3'),
                     ]


    my_title = "Сравнение численного и аналитического решений"
    mv_title_x = "$$x, м$$"
    mv_title_y = "$$c^{\prime}$$"
    file_name = "analit"
    xaxis_min = 0.
    xaxis_max = 300.
    mv_legend_x = 1.
    mv_legend_anchor = "right"

    mv_graphics_stat(graphs_visible, my_title,
                     mv_title_x, mv_title_y,
                     file_name,
                     xaxis_min, xaxis_max,
                     mv_legend_x, mv_legend_anchor)


<center>
<img align="center" src="IMG\analit.png">
</center>

> При очень маленьких значениях $x$ имеется <span class="text-oc">заметное расхождение</span> между численным и аналитическим решением. Это связано с тем, что аналитическое решение соответствует <span class="text-oc">обращению концентрации в бесконечность</span> в точке размещения источника выброса, а численное решение в этой точке принимает <span class="text-oc">конечное значение</span>, обратно пропорциональное шагу сетки по вертикали. 

In [19]:
graphs_visible = [go.Scatter(visible=True, x=uf*UDIN, y=z, 
                               name='{:,.1f} м'.format(z[4]).replace(",", " "),
                               line_dash = 'dot', line_color = '#ffa306')
                 ]


my_title = "Скорость ветра"
mv_title_x = "$$u, м/с$$"
mv_title_y = "$$z,  м$$"
file_name = "diary_tau"
xaxis_min = 0.
xaxis_max = 33.
mv_legend_x = 1.
mv_legend_anchor = "right"

mv_graphics_stat(graphs_visible, my_title,
                 mv_title_x, mv_title_y,
                 file_name,
                 xaxis_min, xaxis_max,
                 mv_legend_x, mv_legend_anchor)

In [20]:
graphs_visible = [go.Scatter(visible=True, x=kf, y=z, 
                               name='{:,.1f} м'.format(z[4]).replace(",", " "),
                               line_dash = 'dot', line_color = '#ffa306')
                 ]


my_title = "Коэффициент турбулентности"
mv_title_x = "$$k, $$"
mv_title_y = "$$z, м$$"
file_name = "diary_tau"
xaxis_min = 0.
xaxis_max = 0.1
mv_legend_x = 1.
mv_legend_anchor = "right"

mv_graphics_stat(graphs_visible, my_title,
                 mv_title_x, mv_title_y,
                 file_name,
                 xaxis_min, xaxis_max,
                 mv_legend_x, mv_legend_anchor)

<center>
<img align="center" src="IMG\thanks.png">
</center>